# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [19]:
view_sentence_range = (201, 220)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 201 to 220:
the grape is your least favorite fruit , but the banana is his least favorite .
china is never warm during summer , but it is usually quiet in november .
she dislikes strawberries , pears , and limes .
his least favorite animal is the mouse .
he dislikes bananas , mangoes , and apples .
the lemon is our favorite fruit , but the orange is my favorite .
france is usually hot during spring , but it is never pleasant in september .
china is chilly during march , and it is never relaxing in may .
france is sometimes mild during november , and it is sometimes busy in spring .
california is usually mild during fall , but it is never chilly in september .
paris is never wonderful during september , and it is hot in july .
we dislike peaches , limes , and oranges .
the apple is your least liked fruit , but the strawberry is th

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of each sentence from `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [20]:
# The sequence_to_sequence_implementation provided in the Course has been extensively 
# used during the development of the Assigment:

def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    # Define the lists:
    source_id_text = []
    target_id_text = []
    
    for source_sentence in source_text.split('\n'):
        source_id_text.append([source_vocab_to_int[word] for word in source_sentence.split()])
        
    for target_sentence in target_text.split('\n'):
        target_id_sentence = [target_vocab_to_int[word] for word in target_sentence.split()]
        target_id_sentence.append(target_vocab_to_int['<EOS>'])
        target_id_text.append(target_id_sentence)
        
    return (source_id_text, target_id_text)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) in [LooseVersion('1.0.0'), LooseVersion('1.0.1')], 'This project requires TensorFlow version 1.0  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1


C:\Users\User\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoding_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.

Return the placeholders in the following the tuple (Input, Targets, Learing Rate, Keep Probability)

In [24]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate, keep probability)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='input')
    targets = tf.placeholder(shape=(None, None), dtype=tf.int32, name='targets')
    learning_rate = tf.placeholder(shape=(None), dtype=tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(shape=(None), dtype=tf.float32, name='keep_prob')
    
    return (inputs, targets, learning_rate, keep_prob) # return the tuple

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoding Input
Implement `process_decoding_input` using TensorFlow to remove the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [25]:
def process_decoding_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for dencoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    decoded_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)
    
    return decoded_input
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_decoding_input(process_decoding_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer using [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn).

In [26]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :return: RNN state
    """
    # TODO: Implement Function
    encoder_cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    # for the use of 'keep_prob' parameter:
    encoder_cell = tf.contrib.rnn.DropoutWrapper(encoder_cell, output_keep_prob=keep_prob)
    encoder = tf.contrib.rnn.MultiRNNCell([encoder_cell] * num_layers)
    _ , encoder_state = tf.nn.dynamic_rnn(encoder, rnn_inputs, dtype=tf.float32)
    
    return encoder_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create training logits using [`tf.contrib.seq2seq.simple_decoder_fn_train()`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_train) and [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder).  Apply the `output_fn` to the [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder) outputs.

In [27]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param sequence_length: Sequence Length
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_fn: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: Train Logits
    """
    # TODO: Implement Function
    decoder_function = tf.contrib.seq2seq.simple_decoder_fn_train(encoder_state)
    # to use the keep_prob:
    decoder_drop = tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob=keep_prob)
    decoder_train, _ , _ = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_drop, decoder_function, inputs=dec_embed_input, 
                                                           sequence_length=sequence_length, scope=decoding_scope) 
    
    Train_logits = output_fn(decoder_train) #create the logits 
    
    return Train_logits

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference logits using [`tf.contrib.seq2seq.simple_decoder_fn_inference()`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_inference) and [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder). 

In [28]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param maximum_length: The maximum allowed time steps to decode
    :param vocab_size: Size of vocabulary
    :param decoding_scope: TensorFlow Variable Scope for decoding
    :param output_fn: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: Inference Logits
    """
    # TODO: Implement Function
    decoder_inference = tf.contrib.seq2seq.simple_decoder_fn_inference(output_fn, encoder_state, dec_embeddings, 
                                                   start_of_sequence_id, end_of_sequence_id, maximum_length-1, 
                                                   vocab_size, dtype=tf.int32)
    # For the inference logits, the dropout is not used
    Inference_Logits, _ , _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, decoder_inference, scope=decoding_scope)
    
    return Inference_Logits

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

- Create RNN cell for decoding using `rnn_size` and `num_layers`.
- Create the output fuction using [`lambda`](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) to transform it's input, logits, to class logits.
- Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob)` function to get the training logits.
- Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, maximum_length, vocab_size, decoding_scope, output_fn, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [29]:
## code adopted from sequence_to_sequence_implementation code provided in the Course:

def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, target_vocab_to_int, keep_prob):
    """
    Create decoding layer
    :param dec_embed_input: Decoder embedded input
    :param dec_embeddings: Decoder embeddings
    :param encoder_state: The encoded state
    :param vocab_size: Size of vocabulary
    :param sequence_length: Sequence Length
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param keep_prob: Dropout keep probability
    :return: Tuple of (Training Logits, Inference Logits)
    """
    # TODO: Implement Function
    cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    dec_cell = tf.contrib.rnn.MultiRNNCell([cell] * num_layers)
    
    with tf.variable_scope("decoding") as decoding_scope:
        output_fn = lambda x: tf.contrib.layers.fully_connected(x, vocab_size, None, scope=decoding_scope)
        Training_Logits = decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob)
        decoding_scope.reuse_variables()
        Inference_Logits = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, target_vocab_to_int['<GO>'],
                                                target_vocab_to_int['<EOS>'], sequence_length, vocab_size,
                                                decoding_scope, output_fn, keep_prob)
        
    return (Training_Logits, Inference_Logits)# tuple of Logits

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Apply embedding to the input data for the encoder.
- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob)`.
- Process target data using your `process_decoding_input(target_data, target_vocab_to_int, batch_size)` function.
- Apply embedding to the target data for the decoder.
- Decode the encoded input using your `decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size, num_layers, target_vocab_to_int, keep_prob)`.

In [30]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size, rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param sequence_length: Sequence Length
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training Logits, Inference Logits)
    """
    # TODO: Implement Function
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, enc_embedding_size)

    encoder_state = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob)
    
    processed_target_data = process_decoding_input(target_data, target_vocab_to_int, batch_size)
    
    dec_embedding = tf.Variable(tf.random_uniform([target_vocab_size, dec_embedding_size]))
    dec_embed = tf.nn.embedding_lookup(dec_embedding, processed_target_data)
    Training_Logits, Inference_Logits = decoding_layer(dec_embed, dec_embedding,
                                                       encoder_state, target_vocab_size, sequence_length, rnn_size,
                                                       num_layers, target_vocab_to_int, keep_prob)
    return (Training_Logits, Inference_Logits)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability

In [31]:
# try different sets:

# Number of Epochs
epochs = 10 # 20, 50
# Batch Size
batch_size = 128 #100, 200, 256
# RNN Size
rnn_size = 64
# Number of Layers
num_layers = 2 # 3 layer also tested
# Embedding Size
encoding_embedding_size = 256 # a power of 2 (50,200, 300 tested also)
decoding_embedding_size = 256
# Learning Rate
learning_rate = 0.001 #0.01
# Dropout Keep Probability
keep_probability = 0.80 #0.70,0.75,0.85

### Build the Graph
Build the graph using the neural network you implemented.

In [32]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_source_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob = model_inputs()
    sequence_length = tf.placeholder_with_default(max_source_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)
    
    train_logits, inference_logits = seq2seq_model(
        tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(source_vocab_to_int), len(target_vocab_to_int),
        encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, target_vocab_to_int)

    tf.identity(inference_logits, 'logits')
    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            train_logits,
            targets,
            tf.ones([input_shape[0], sequence_length]))

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [33]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import time

def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1]), (0,0)],
            'constant')

    return np.mean(np.equal(target, np.argmax(logits, 2)))

train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]

valid_source = helper.pad_sentence_batch(source_int_text[:batch_size])
valid_target = helper.pad_sentence_batch(target_int_text[:batch_size])

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch) in enumerate(
                helper.batch_data(train_source, train_target, batch_size)):
            start_time = time.time()
            
            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 sequence_length: target_batch.shape[1],
                 keep_prob: keep_probability})
            
            batch_train_logits = sess.run(
                inference_logits,
                {input_data: source_batch, keep_prob: 1.0})
            batch_valid_logits = sess.run(
                inference_logits,
                {input_data: valid_source, keep_prob: 1.0})
                
            train_acc = get_accuracy(target_batch, batch_train_logits)
            valid_acc = get_accuracy(np.array(valid_target), batch_valid_logits)
            end_time = time.time()
            print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.3f}, Validation Accuracy: {:>6.3f}, Loss: {:>6.3f}'
                  .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    0/1077 - Train Accuracy:  0.295, Validation Accuracy:  0.305, Loss:  5.887
Epoch   0 Batch    1/1077 - Train Accuracy:  0.223, Validation Accuracy:  0.307, Loss:  5.817
Epoch   0 Batch    2/1077 - Train Accuracy:  0.208, Validation Accuracy:  0.307, Loss:  5.767
Epoch   0 Batch    3/1077 - Train Accuracy:  0.241, Validation Accuracy:  0.307, Loss:  5.688
Epoch   0 Batch    4/1077 - Train Accuracy:  0.226, Validation Accuracy:  0.307, Loss:  5.637
Epoch   0 Batch    5/1077 - Train Accuracy:  0.263, Validation Accuracy:  0.307, Loss:  5.551
Epoch   0 Batch    6/1077 - Train Accuracy:  0.243, Validation Accuracy:  0.306, Loss:  5.499
Epoch   0 Batch    7/1077 - Train Accuracy:  0.227, Validation Accuracy:  0.306, Loss:  5.445
Epoch   0 Batch    8/1077 - Train Accuracy:  0.233, Validation Accuracy:  0.305, Loss:  5.373
Epoch   0 Batch    9/1077 - Train Accuracy:  0.236, Validation Accuracy:  0.305, Loss:  5.285
Epoch   0 Batch   10/1077 - Train Accuracy:  0.198, Validati

Epoch   0 Batch   88/1077 - Train Accuracy:  0.332, Validation Accuracy:  0.388, Loss:  3.026
Epoch   0 Batch   89/1077 - Train Accuracy:  0.324, Validation Accuracy:  0.395, Loss:  3.042
Epoch   0 Batch   90/1077 - Train Accuracy:  0.326, Validation Accuracy:  0.397, Loss:  3.064
Epoch   0 Batch   91/1077 - Train Accuracy:  0.392, Validation Accuracy:  0.397, Loss:  2.786
Epoch   0 Batch   92/1077 - Train Accuracy:  0.352, Validation Accuracy:  0.397, Loss:  2.963
Epoch   0 Batch   93/1077 - Train Accuracy:  0.337, Validation Accuracy:  0.397, Loss:  3.027
Epoch   0 Batch   94/1077 - Train Accuracy:  0.351, Validation Accuracy:  0.395, Loss:  2.935
Epoch   0 Batch   95/1077 - Train Accuracy:  0.372, Validation Accuracy:  0.397, Loss:  2.867
Epoch   0 Batch   96/1077 - Train Accuracy:  0.357, Validation Accuracy:  0.398, Loss:  2.938
Epoch   0 Batch   97/1077 - Train Accuracy:  0.369, Validation Accuracy:  0.401, Loss:  2.911
Epoch   0 Batch   98/1077 - Train Accuracy:  0.388, Validati

Epoch   0 Batch  176/1077 - Train Accuracy:  0.349, Validation Accuracy:  0.430, Loss:  2.682
Epoch   0 Batch  177/1077 - Train Accuracy:  0.356, Validation Accuracy:  0.434, Loss:  2.734
Epoch   0 Batch  178/1077 - Train Accuracy:  0.397, Validation Accuracy:  0.440, Loss:  2.563
Epoch   0 Batch  179/1077 - Train Accuracy:  0.374, Validation Accuracy:  0.439, Loss:  2.697
Epoch   0 Batch  180/1077 - Train Accuracy:  0.373, Validation Accuracy:  0.431, Loss:  2.617
Epoch   0 Batch  181/1077 - Train Accuracy:  0.388, Validation Accuracy:  0.435, Loss:  2.612
Epoch   0 Batch  182/1077 - Train Accuracy:  0.393, Validation Accuracy:  0.437, Loss:  2.527
Epoch   0 Batch  183/1077 - Train Accuracy:  0.350, Validation Accuracy:  0.428, Loss:  2.684
Epoch   0 Batch  184/1077 - Train Accuracy:  0.392, Validation Accuracy:  0.440, Loss:  2.605
Epoch   0 Batch  185/1077 - Train Accuracy:  0.384, Validation Accuracy:  0.440, Loss:  2.620
Epoch   0 Batch  186/1077 - Train Accuracy:  0.376, Validati

Epoch   0 Batch  264/1077 - Train Accuracy:  0.412, Validation Accuracy:  0.474, Loss:  2.400
Epoch   0 Batch  265/1077 - Train Accuracy:  0.432, Validation Accuracy:  0.475, Loss:  2.311
Epoch   0 Batch  266/1077 - Train Accuracy:  0.449, Validation Accuracy:  0.468, Loss:  2.236
Epoch   0 Batch  267/1077 - Train Accuracy:  0.433, Validation Accuracy:  0.468, Loss:  2.240
Epoch   0 Batch  268/1077 - Train Accuracy:  0.447, Validation Accuracy:  0.477, Loss:  2.318
Epoch   0 Batch  269/1077 - Train Accuracy:  0.399, Validation Accuracy:  0.479, Loss:  2.474
Epoch   0 Batch  270/1077 - Train Accuracy:  0.384, Validation Accuracy:  0.472, Loss:  2.461
Epoch   0 Batch  271/1077 - Train Accuracy:  0.423, Validation Accuracy:  0.466, Loss:  2.314
Epoch   0 Batch  272/1077 - Train Accuracy:  0.442, Validation Accuracy:  0.473, Loss:  2.266
Epoch   0 Batch  273/1077 - Train Accuracy:  0.436, Validation Accuracy:  0.474, Loss:  2.247
Epoch   0 Batch  274/1077 - Train Accuracy:  0.434, Validati

Epoch   0 Batch  352/1077 - Train Accuracy:  0.418, Validation Accuracy:  0.497, Loss:  2.223
Epoch   0 Batch  353/1077 - Train Accuracy:  0.430, Validation Accuracy:  0.496, Loss:  2.284
Epoch   0 Batch  354/1077 - Train Accuracy:  0.425, Validation Accuracy:  0.484, Loss:  2.158
Epoch   0 Batch  355/1077 - Train Accuracy:  0.457, Validation Accuracy:  0.490, Loss:  2.080
Epoch   0 Batch  356/1077 - Train Accuracy:  0.455, Validation Accuracy:  0.492, Loss:  2.109
Epoch   0 Batch  357/1077 - Train Accuracy:  0.490, Validation Accuracy:  0.496, Loss:  1.943
Epoch   0 Batch  358/1077 - Train Accuracy:  0.425, Validation Accuracy:  0.490, Loss:  2.174
Epoch   0 Batch  359/1077 - Train Accuracy:  0.445, Validation Accuracy:  0.479, Loss:  2.083
Epoch   0 Batch  360/1077 - Train Accuracy:  0.453, Validation Accuracy:  0.500, Loss:  2.110
Epoch   0 Batch  361/1077 - Train Accuracy:  0.441, Validation Accuracy:  0.498, Loss:  2.169
Epoch   0 Batch  362/1077 - Train Accuracy:  0.461, Validati

...

Epoch   9 Batch  700/1077 - Train Accuracy:  0.946, Validation Accuracy:  0.910, Loss:  0.079
Epoch   9 Batch  701/1077 - Train Accuracy:  0.929, Validation Accuracy:  0.918, Loss:  0.089
Epoch   9 Batch  702/1077 - Train Accuracy:  0.918, Validation Accuracy:  0.921, Loss:  0.108
Epoch   9 Batch  703/1077 - Train Accuracy:  0.936, Validation Accuracy:  0.914, Loss:  0.086
Epoch   9 Batch  704/1077 - Train Accuracy:  0.908, Validation Accuracy:  0.917, Loss:  0.110
Epoch   9 Batch  705/1077 - Train Accuracy:  0.912, Validation Accuracy:  0.914, Loss:  0.099
Epoch   9 Batch  706/1077 - Train Accuracy:  0.886, Validation Accuracy:  0.915, Loss:  0.119
Epoch   9 Batch  707/1077 - Train Accuracy:  0.942, Validation Accuracy:  0.911, Loss:  0.081
Epoch   9 Batch  708/1077 - Train Accuracy:  0.911, Validation Accuracy:  0.911, Loss:  0.095
Epoch   9 Batch  709/1077 - Train Accuracy:  0.918, Validation Accuracy:  0.906, Loss:  0.091
Epoch   9 Batch  710/1077 - Train Accuracy:  0.909, Validati

Epoch   9 Batch  788/1077 - Train Accuracy:  0.936, Validation Accuracy:  0.922, Loss:  0.074
Epoch   9 Batch  789/1077 - Train Accuracy:  0.921, Validation Accuracy:  0.918, Loss:  0.091
Epoch   9 Batch  790/1077 - Train Accuracy:  0.868, Validation Accuracy:  0.923, Loss:  0.093
Epoch   9 Batch  791/1077 - Train Accuracy:  0.913, Validation Accuracy:  0.926, Loss:  0.098
Epoch   9 Batch  792/1077 - Train Accuracy:  0.926, Validation Accuracy:  0.924, Loss:  0.099
Epoch   9 Batch  793/1077 - Train Accuracy:  0.948, Validation Accuracy:  0.923, Loss:  0.072
Epoch   9 Batch  794/1077 - Train Accuracy:  0.907, Validation Accuracy:  0.918, Loss:  0.080
Epoch   9 Batch  795/1077 - Train Accuracy:  0.923, Validation Accuracy:  0.918, Loss:  0.105
Epoch   9 Batch  796/1077 - Train Accuracy:  0.942, Validation Accuracy:  0.919, Loss:  0.081
Epoch   9 Batch  797/1077 - Train Accuracy:  0.934, Validation Accuracy:  0.915, Loss:  0.082
Epoch   9 Batch  798/1077 - Train Accuracy:  0.910, Validati

Epoch   9 Batch  876/1077 - Train Accuracy:  0.945, Validation Accuracy:  0.914, Loss:  0.081
Epoch   9 Batch  877/1077 - Train Accuracy:  0.935, Validation Accuracy:  0.920, Loss:  0.079
Epoch   9 Batch  878/1077 - Train Accuracy:  0.929, Validation Accuracy:  0.920, Loss:  0.082
Epoch   9 Batch  879/1077 - Train Accuracy:  0.937, Validation Accuracy:  0.915, Loss:  0.078
Epoch   9 Batch  880/1077 - Train Accuracy:  0.933, Validation Accuracy:  0.913, Loss:  0.082
Epoch   9 Batch  881/1077 - Train Accuracy:  0.923, Validation Accuracy:  0.913, Loss:  0.094
Epoch   9 Batch  882/1077 - Train Accuracy:  0.936, Validation Accuracy:  0.914, Loss:  0.085
Epoch   9 Batch  883/1077 - Train Accuracy:  0.928, Validation Accuracy:  0.913, Loss:  0.117
Epoch   9 Batch  884/1077 - Train Accuracy:  0.930, Validation Accuracy:  0.911, Loss:  0.076
Epoch   9 Batch  885/1077 - Train Accuracy:  0.944, Validation Accuracy:  0.907, Loss:  0.064
Epoch   9 Batch  886/1077 - Train Accuracy:  0.929, Validati

Epoch   9 Batch  964/1077 - Train Accuracy:  0.968, Validation Accuracy:  0.931, Loss:  0.065
Epoch   9 Batch  965/1077 - Train Accuracy:  0.917, Validation Accuracy:  0.928, Loss:  0.087
Epoch   9 Batch  966/1077 - Train Accuracy:  0.927, Validation Accuracy:  0.924, Loss:  0.071
Epoch   9 Batch  967/1077 - Train Accuracy:  0.927, Validation Accuracy:  0.912, Loss:  0.081
Epoch   9 Batch  968/1077 - Train Accuracy:  0.891, Validation Accuracy:  0.911, Loss:  0.109
Epoch   9 Batch  969/1077 - Train Accuracy:  0.924, Validation Accuracy:  0.911, Loss:  0.102
Epoch   9 Batch  970/1077 - Train Accuracy:  0.947, Validation Accuracy:  0.913, Loss:  0.087
Epoch   9 Batch  971/1077 - Train Accuracy:  0.935, Validation Accuracy:  0.913, Loss:  0.084
Epoch   9 Batch  972/1077 - Train Accuracy:  0.937, Validation Accuracy:  0.926, Loss:  0.077
Epoch   9 Batch  973/1077 - Train Accuracy:  0.941, Validation Accuracy:  0.932, Loss:  0.065
Epoch   9 Batch  974/1077 - Train Accuracy:  0.939, Validati

Epoch   9 Batch 1052/1077 - Train Accuracy:  0.945, Validation Accuracy:  0.924, Loss:  0.082
Epoch   9 Batch 1053/1077 - Train Accuracy:  0.938, Validation Accuracy:  0.915, Loss:  0.083
Epoch   9 Batch 1054/1077 - Train Accuracy:  0.941, Validation Accuracy:  0.909, Loss:  0.074
Epoch   9 Batch 1055/1077 - Train Accuracy:  0.938, Validation Accuracy:  0.909, Loss:  0.084
Epoch   9 Batch 1056/1077 - Train Accuracy:  0.933, Validation Accuracy:  0.914, Loss:  0.079
Epoch   9 Batch 1057/1077 - Train Accuracy:  0.943, Validation Accuracy:  0.909, Loss:  0.092
Epoch   9 Batch 1058/1077 - Train Accuracy:  0.925, Validation Accuracy:  0.914, Loss:  0.089
Epoch   9 Batch 1059/1077 - Train Accuracy:  0.914, Validation Accuracy:  0.911, Loss:  0.111
Epoch   9 Batch 1060/1077 - Train Accuracy:  0.940, Validation Accuracy:  0.925, Loss:  0.075
Epoch   9 Batch 1061/1077 - Train Accuracy:  0.925, Validation Accuracy:  0.925, Loss:  0.102
Epoch   9 Batch 1062/1077 - Train Accuracy:  0.933, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [34]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [35]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [36]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    # Convert the sentence to lowercase:
    sentence = sentence.lower()
    word_ids = [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in sentence.split()]
    
    return word_ids

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [37]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('logits:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence], keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in np.argmax(translate_logits, 1)]))
print('  French Words: {}'.format([target_int_to_vocab[i] for i in np.argmax(translate_logits, 1)]))

Input
  Word Ids:      [102, 62, 156, 178, 220, 208, 60]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [172, 149, 280, 253, 111, 28, 58, 119, 1]
  French Words: ['il', 'a', 'vu', 'un', 'vieux', 'camion', 'jaune', '.', '<EOS>']


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.